In [2]:
from collections import OrderedDict
from fastprogress import progress_bar
from pathlib import Path
from sklearn.model_selection import train_test_split, ShuffleSplit
from torch import nn
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import functools
import os
import pandas as pd
import random
import shutil
import torch
import torch.nn.functional as F


target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
input_cols = ['sequence', 'structure', 'predicted_loop_type']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_Mg_50C', 'deg_error_pH10', 'deg_error_50C']

token_dicts = {
    "sequence": {x: i for i, x in enumerate("ACGU")},
    "structure": {x: i for i, x in enumerate('().')},
    "predicted_loop_type": {x: i for i, x in enumerate("BEHIMSX")}
}


def set_seed(seed: int = 12042):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [4]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from torch import nn
from torch.utils.data import Dataset

import functools


BASE_PATH = "/kaggle/input/stanford-covid-vaccine"
MODEL_SAVE_PATH = "/kaggle/model"


def preprocess_inputs(df, cols):
    return np.concatenate([preprocess_feature_col(df, col) for col in cols], axis=2)


def preprocess_feature_col(df, col):
    dic = token_dicts[col]
    dic_len = len(dic)
    seq_length = len(df[col][0])
    ident = np.identity(dic_len)
    # convert to one hot
    arr = np.array(
        df[[col]].applymap(lambda seq: [ident[dic[x]] for x in seq]).values.tolist()
    ).squeeze(1)
    # shape: data_size x seq_length x dic_length
    assert arr.shape == (len(df), seq_length, dic_len)
    return arr


def preprocess(base_data, is_test=False):
    inputs = preprocess_inputs(base_data, input_cols)
    if is_test:
        labels = None
    else:
        labels = np.array(base_data[target_cols].values.tolist()).transpose((0, 2, 1))
        assert labels.shape[2] == len(target_cols)
    assert inputs.shape[2] == 14
    return inputs, labels


def get_bpp_feature(bpp):
    bpp_nb_mean = 0.077522  # mean of bpps_nb across all training data
    bpp_nb_std = 0.08914  # std of bpps_nb across all training data
    bpp_max = bpp.max(-1)[0]
    bpp_sum = bpp.sum(-1)
    bpp_nb = torch.true_divide((bpp > 0).sum(dim=1), bpp.shape[1])
    bpp_nb = torch.true_divide(bpp_nb - bpp_nb_mean, bpp_nb_std)
    return [bpp_max.unsqueeze(2), bpp_sum.unsqueeze(2), bpp_nb.unsqueeze(2)]


@functools.lru_cache(5000)
def load_from_id(id_):
    path = Path(BASE_PATH) / f"bpps/{id_}.npy"
    data = np.load(str(path))
    return data


def get_distance_matrix(leng):
    idx = np.arange(leng)
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1 / Ds
    Ds = Ds[None, :, :]
    Ds = np.repeat(Ds, 1, axis=0)

    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis=3)
    print(Ds.shape)
    return Ds


def get_structure_adj(df):
    Ss = []
    for i in range(len(df)):
        seq_length = df["seq_length"].iloc[i]
        structure = df["structure"].iloc[i]
        sequence = df["sequence"].iloc[i]

        cue = []
        a_structures = OrderedDict([
            (("A", "U"), np.zeros([seq_length, seq_length])),
            (("C", "G"), np.zeros([seq_length, seq_length])),
            (("U", "G"), np.zeros([seq_length, seq_length])),
            (("U", "A"), np.zeros([seq_length, seq_length])),
            (("G", "C"), np.zeros([seq_length, seq_length])),
            (("G", "U"), np.zeros([seq_length, seq_length])),
        ])
        for j in range(seq_length):
            if structure[j] == "(":
                cue.append(j)
            elif structure[j] == ")":
                start = cue.pop()
                a_structures[(sequence[start], sequence[j])][start, j] = 1
                a_structures[(sequence[j], sequence[start])][j, start] = 1

        a_strc = np.stack([a for a in a_structures.values()], axis=2)
        a_strc = np.sum(a_strc, axis=2, keepdims=True)
        Ss.append(a_strc)

    Ss = np.array(Ss)
    return Ss


def create_loader(df, batch_size=1, is_test=False):
    features, labels = preprocess(df, is_test)
    features_tensor = torch.from_numpy(features)
    if labels is not None:
        labels_tensor = torch.from_numpy(labels)
        dataset = VacDataset(features_tensor, df, labels_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, drop_last=False)
    else:
        dataset = VacDataset(features_tensor, df, None)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=False, drop_last=False)
    return loader


class VacDataset(Dataset):
    def __init__(self, features, df, labels=None):
        self.features = features
        self.labels = labels
        self.test = labels is None
        self.ids = df["id"]
        self.score = None
        self.structure_adj = get_structure_adj(df)
        self.distance_matrix = get_distance_matrix(self.structure_adj.shape[1])
        if "score" in df.columns:
            self.score = df["score"]
        else:
            df["score"] = 1.0
            self.score = df["score"]
        self.signal_to_noise = None
        if not self.test:
            self.signal_to_noise = df["signal_to_noise"]
            assert self.features.shape[0] == self.labels.shape[0]
        else:
            assert self.ids is not None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        bpp = torch.from_numpy(load_from_id(self.ids[index]).copy()).float()
        adj = self.structure_adj[index]
        distance = self.distance_matrix[0]
        bpp = np.concatenate([bpp[:, :, None], adj, distance], axis=2)
        if self.test:
            return dict(sequence=self.features[index].float(), bpp=bpp, ids=self.ids[index])
        else:
            return dict(sequence=self.features[index].float(), bpp=bpp,
                        label=self.labels[index], ids=self.ids[index],
                        signal_to_noise=self.signal_to_noise[index],
                        score=self.score[index])


### Model

In [5]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer

import math


class Conv1dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv1dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv1d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class Conv2dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv2dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class SeqEncoder(nn.Module):
    def __init__(self, in_dim: int):
        super(SeqEncoder, self).__init__()
        self.conv0 = Conv1dStack(in_dim, 128, 3, padding=1)
        self.conv1 = Conv1dStack(128, 64, 6, padding=5, dilation=2)
        self.conv2 = Conv1dStack(64, 32, 15, padding=7, dilation=1)
        self.conv3 = Conv1dStack(32, 32, 30, padding=29, dilation=2)

    def forward(self, x):
        x1 = self.conv0(x)
        x2 = self.conv1(x1)
        x3 = self.conv2(x2)
        x4 = self.conv3(x3)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        # x = x.permute(0, 2, 1).contiguous()
        # BATCH x 256 x seq_length
        return x


class BppAttn(nn.Module):
    def __init__(self, in_channel: int, out_channel: int):
        super(BppAttn, self).__init__()
        self.conv0 = Conv1dStack(in_channel, out_channel, 3, padding=1)
        self.bpp_conv = Conv2dStack(5, out_channel)

    def forward(self, x, bpp):
        x = self.conv0(x)
        bpp = self.bpp_conv(bpp)
        # BATCH x C x SEQ x SEQ
        # BATCH x C x SEQ
        x = torch.matmul(bpp, x.unsqueeze(-1))
        return x.squeeze(-1)


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=143):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerWrapper(nn.Module):
    def __init__(self, dmodel=256, nhead=8, num_layers=2):
        super(TransformerWrapper, self).__init__()
        self.pos_encoder = PositionalEncoding(256)
        encoder_layer = TransformerEncoderLayer(d_model=dmodel, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers)
        self.pos_emb = PositionalEncoding(dmodel)

    def flatten_parameters(self):
        pass

    def forward(self, x):
        x = x.permute((1, 0, 2)).contiguous()
        x = self.pos_emb(x)
        x = self.transformer_encoder(x)
        x = x.permute((1, 0, 2)).contiguous()
        return x, None


class RnnLayers(nn.Module):
    def __init__(self, dmodel, dropout=0.3, transformer_layers: int = 2):
        super(RnnLayers, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.rnn0 = TransformerWrapper(dmodel, nhead=8, num_layers=transformer_layers)
        self.rnn1 = nn.LSTM(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)
        self.rnn2 = nn.GRU(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)

    def forward(self, x):
        self.rnn0.flatten_parameters()
        x, _ = self.rnn0(x)
        if self.rnn1 is not None:
            self.rnn1.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn1(x)
        if self.rnn2 is not None:
            self.rnn2.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn2(x)
        return x

    
class BaseAttnModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(BaseAttnModel, self).__init__()
        self.linear0 = nn.Linear(14 + 3, 1)
        self.seq_encoder_x = SeqEncoder(18)
        self.attn = BppAttn(256, 128)
        self.seq_encoder_bpp = SeqEncoder(128)
        self.seq = RnnLayers(256 * 2, dropout=0.3,
                             transformer_layers=transformer_layers)

    def forward(self, x, bpp):
        bpp_features = get_bpp_feature(bpp[:, :, :, 0].float())
        x = torch.cat([x] + bpp_features, dim=-1)
        learned = self.linear0(x)
        x = torch.cat([x, learned], dim=-1)
        x = x.permute(0, 2, 1).contiguous().float()
        # BATCH x 18 x seq_len
        bpp = bpp.permute([0, 3, 1, 2]).contiguous().float()
        # BATCH x 5 x seq_len x seq_len
        x = self.seq_encoder_x(x)
        # BATCH x 256 x seq_len
        bpp = self.attn(x, bpp)
        bpp = self.seq_encoder_bpp(bpp)
        # BATCH x 256 x seq_len
        x = x.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        bpp = bpp.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        x = torch.cat([x, bpp], dim=2)
        # BATCH x seq_len x 512
        x = self.seq(x)
        return x


class AEModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(AEModel, self).__init__()
        self.seq = BaseAttnModel(transformer_layers=transformer_layers)
        self.linear = nn.Sequential(
            nn.Linear(256 * 2, 14),
            nn.Sigmoid(),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = F.dropout(x, p=0.3)
        x = self.linear(x)
        return x


class FromAeModel(nn.Module):
    def __init__(self, seq, pred_len=68, dmodel: int = 256):
        super(FromAeModel, self).__init__()
        self.seq = seq
        self.pred_len = pred_len
        self.linear = nn.Sequential(
            nn.Linear(dmodel * 2, len(target_cols)),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = self.linear(x)
        x = x[:, :self.pred_len]
        return x


In [6]:
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_train_data.head()

device = torch.device('cuda')
BATCH_SIZE = 64
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()

features, _ = preprocess(base_train_data, True)
features_tensor = torch.from_numpy(features)
dataset0 = VacDataset(features_tensor, base_train_data, None)
features, _ = preprocess(public_df, True)
features_tensor = torch.from_numpy(features)
dataset1 = VacDataset(features_tensor, public_df, None)
features, _ = preprocess(private_df, True)
features_tensor = torch.from_numpy(features)
dataset2 = VacDataset(features_tensor, private_df, None)

loader0 = torch.utils.data.DataLoader(dataset0, BATCH_SIZE, shuffle=False, drop_last=False)
loader1 = torch.utils.data.DataLoader(dataset1, BATCH_SIZE, shuffle=False, drop_last=False)
loader2 = torch.utils.data.DataLoader(dataset2, BATCH_SIZE, shuffle=False, drop_last=False)

public_df: (629, 7)
private_df: (3005, 7)
(1, 107, 107, 3)
(1, 107, 107, 3)
(1, 130, 130, 3)


### Pretrain

In [7]:
def learn_from_batch_ae(model, data, device):
    seq = data["sequence"].clone()
    seq[:, :, :14] = F.dropout2d(seq[:, :, :14], p=0.3)
    target = data["sequence"][:, :, :14]
    out = model(seq.to(device), data["bpp"].to(device))
    loss = F.binary_cross_entropy(out, target.to(device))
    return loss


def train_ae(model, train_data, optimizer, lr_scheduler, epochs=10, device="cpu",
             start_epoch: int = 0, start_it: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    it = start_it
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    min_loss = 10.0
    min_loss_epoch = 0
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            optimizer.zero_grad()
            loss = learn_from_batch_ae(model, data, device)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            if lr_scheduler:
                lr_scheduler.step()
            loss_v = loss.item()
            losses.append(loss_v)
            it += 1
        loss_m = np.mean(losses)
        if loss_m < min_loss:
            min_loss_epoch = epoch
            min_loss = loss_m
        print(f'epoch: {epoch} loss: {loss_m}')
        losses = []
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    return dict(end_epoch=end_epoch, it=it, min_loss_epoch=min_loss_epoch)


In [8]:
import shutil


set_seed(123)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)

lr_scheduler = None
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AEModel()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
res = dict(end_epoch=0, it=0, min_loss_epoch=0)
epochs = [5, 5, 5, 5]
for e in epochs:
    res = train_ae(model, loader0, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    res = train_ae(model, loader1, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    res = train_ae(model, loader2, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])

epoch = res["min_loss_epoch"]
shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"model-{epoch}.pt"), "ae-model.pt")

device: cuda


epoch: 0
epoch: 0 loss: 0.32876423275784444
epoch: 1
epoch: 1 loss: 0.17384451274809085
epoch: 2
epoch: 2 loss: 0.13517996374713748
epoch: 3
epoch: 3 loss: 0.09794114353625398
epoch: 4
epoch: 4 loss: 0.06799206647433732
device: cuda


epoch: 5
epoch: 5 loss: 0.0417643778026104
epoch: 6
epoch: 6 loss: 0.034109911695122716
epoch: 7
epoch: 7 loss: 0.028408176824450494
epoch: 8
epoch: 8 loss: 0.024150612019002437
epoch: 9
epoch: 9 loss: 0.02263587936758995
device: cuda


epoch: 10
epoch: 10 loss: 0.02586393327789104
epoch: 11
epoch: 11 loss: 0.01965850052681375
epoch: 12
epoch: 12 loss: 0.01738638488298401
epoch: 13
epoch: 13 loss: 0.015894609226032773
epoch: 14
epoch: 14 loss: 0.015304847382643122
device: cuda


epoch: 15
epoch: 15 loss: 0.010744662195640175
epoch: 16
epoch: 16 loss: 0.009451158197694704
epoch: 17
epoch: 17 loss: 0.076314049711647
epoch: 18
epoch: 18 loss: 0.05028964422251049
epoch: 19
epoch: 19 loss: 0.0151426939685878
device: cuda


...

epoch: 40
epoch: 40 loss: 0.012750162822293474
epoch: 41
epoch: 41 loss: 0.011744047257494419
epoch: 42
epoch: 42 loss: 0.011719081590467311
epoch: 43
epoch: 43 loss: 0.011909238577998698
epoch: 44
epoch: 44 loss: 0.01125228791398571
device: cuda


epoch: 45
epoch: 45 loss: 0.007034959112245001
epoch: 46
epoch: 46 loss: 0.006906548649758885
epoch: 47
epoch: 47 loss: 0.0065248815808445215
epoch: 48
epoch: 48 loss: 0.005990852454775258
epoch: 49
epoch: 49 loss: 0.006052771791521656
device: cuda


epoch: 50
epoch: 50 loss: 0.004581833491101861
epoch: 51
epoch: 51 loss: 0.004607548704370856
epoch: 52
epoch: 52 loss: 0.004300079471431672
epoch: 53
epoch: 53 loss: 0.004429742484353483
epoch: 54
epoch: 54 loss: 0.004295563488267362
device: cuda


epoch: 55
epoch: 55 loss: 0.011128994497530003
epoch: 56
epoch: 56 loss: 0.010216735778970922
epoch: 57
epoch: 57 loss: 0.009866123365119417
epoch: 58
epoch: 58 loss: 0.009493855600978466
epoch: 59
epoch: 59 loss: 0.009372916984114242


'ae-model.pt'

### Training

In [9]:
def MCRMSE(y_true, y_pred):
    colwise_mse = torch.mean(torch.square(y_true - y_pred), dim=1)
    return torch.mean(torch.sqrt(colwise_mse), dim=1)


def sn_mcrmse_loss(predict, target, signal_to_noise):
    loss = MCRMSE(target, predict)
    weight = 0.5 * torch.log(signal_to_noise + 1.01)
    loss = (loss * weight).mean()
    return loss


def learn_from_batch(model, data, optimizer, lr_scheduler, device):
    optimizer.zero_grad()
    out = model(data["sequence"].to(device), data["bpp"].to(device))
    signal_to_noise = data["signal_to_noise"] * data["score"]
    loss = sn_mcrmse_loss(out, data["label"].to(device), signal_to_noise.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    if lr_scheduler:
        lr_scheduler.step()
    return out, loss


def evaluate(model, valid_data, device):
    model.eval()
    loss_list = []
    mcrmse = []
    for i, data in enumerate(valid_data):
        with torch.no_grad():
            y = model(data["sequence"].to(device), data["bpp"].to(device))
            mcrmse_ = MCRMSE(data["label"].to(device), y)[data["signal_to_noise"] > 1]
            mcrmse.append(mcrmse_.mean().item())
            loss = sn_mcrmse_loss(y, data["label"].to(device), data["signal_to_noise"].to(device))
            loss_list.append(loss.item())
    model.train()
    return dict(loss=np.mean(loss_list), mcmse=np.mean(mcrmse))


def train(model, train_data, valid_data, optimizer, lr_scheduler, epochs=10, device="cpu",
          start_epoch: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    writer = SummaryWriter(log_path)
    it = 0
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    min_eval_loss = 10.0
    min_eval_epoch = None
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            _, loss = learn_from_batch(model, data, optimizer, lr_scheduler, device)
            loss_v = loss.item()
            writer.add_scalar('loss', loss_v, it)
            losses.append(loss_v)
            it += 1
        print(f'epoch: {epoch} loss: {np.mean(losses)}')
        losses = []

        eval_result = evaluate(model, valid_data, device)
        eval_loss = eval_result["loss"]
        if eval_loss <= min_eval_loss:
            min_eval_epoch = epoch
            min_eval_loss = eval_loss

        print(f"eval loss: {eval_loss} {eval_result['mcmse']}")
        writer.add_scalar(f"evaluate/loss", eval_loss, epoch)
        writer.add_scalar(f"evaluate/mcmse", eval_result["mcmse"], epoch)
        model.train()
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    print(f'min eval loss: {min_eval_loss} epoch {min_eval_epoch}')
    return min_eval_epoch


In [10]:
device = torch.device('cuda')
BATCH_SIZE = 64
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
samples = base_train_data
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
split = ShuffleSplit(n_splits=5, test_size=.1)
ids = samples.reset_index()["id"]
set_seed(124)
for fold, (train_index, test_index) in enumerate(split.split(samples)):
    print(f"fold: {fold}")
    train_df = samples.loc[train_index].reset_index()
    val_df = samples.loc[test_index].reset_index()
    train_loader = create_loader(train_df, BATCH_SIZE)
    valid_loader = create_loader(val_df, BATCH_SIZE)
    print(train_df.shape, val_df.shape)
    ae_model = AEModel()
    state_dict = torch.load("./ae-model.pt")
    ae_model.load_state_dict(state_dict)
    del state_dict
    model = FromAeModel(ae_model.seq)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    lr_scheduler = None
    epoch = train(model, train_loader, valid_loader, optimizer, lr_scheduler, 200, device=device,
                  log_path=f"logs/{fold}")
    shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"./model-{epoch}.pt"), f"model_prediction/model-{fold}.pt")
    del model

fold: 0
(1, 107, 107, 3)
(1, 107, 107, 3)
(2160, 21) (240, 21)
device: cuda


epoch: 0
epoch: 0 loss: 0.30328408245117844
eval loss: 0.24477458371463193 0.3203635233013302
epoch: 1
epoch: 1 loss: 0.23992183833140857
eval loss: 0.22442239347904497 0.29474035816609157
epoch: 2
epoch: 2 loss: 0.22457168326662533
eval loss: 0.2173772916021977 0.28430703589376366
epoch: 3
epoch: 3 loss: 0.21733221241221168
eval loss: 0.21192233054796794 0.27854443619254815
epoch: 4
epoch: 4 loss: 0.20716392028672623
eval loss: 0.19962869874609607 0.26171297547913047
epoch: 5
epoch: 5 loss: 0.20064288939980898
eval loss: 0.19585818323394455 0.25810428234856475
epoch: 6
epoch: 6 loss: 0.19515951821286712
eval loss: 0.19228525378048794 0.25202136063811853
epoch: 7
epoch: 7 loss: 0.1885009006629958
eval loss: 0.18490994429826302 0.2431887897048674
epoch: 8
epoch: 8 loss: 0.1843454202390462
eval loss: 0.18012231510078838 0.23667789439978196
epoch: 9
epoch: 9 loss: 0.18167793555127096
eval loss: 0.18204977036604536 0.2392055907432064
epoch: 10
epoch: 10 loss: 0.17877064091649225
eval loss:

epoch: 0
epoch: 0 loss: 0.3059304102187981
eval loss: 0.2759530584339495 0.3537633591778523
epoch: 1
epoch: 1 loss: 0.2451929598643411
eval loss: 0.24158802606711932 0.31074517993202827
epoch: 2
epoch: 2 loss: 0.22849072834381376
eval loss: 0.22959801051613335 0.29473569902766766
epoch: 3
epoch: 3 loss: 0.2174535215956561
eval loss: 0.2164551585498095 0.2784676319772002
epoch: 4
epoch: 4 loss: 0.20784390787197865
eval loss: 0.20924057224446732 0.2686137020170599
epoch: 5
epoch: 5 loss: 0.2008919564085554
eval loss: 0.20521493839760896 0.26446654492847177
epoch: 6
epoch: 6 loss: 0.19533942209906907
eval loss: 0.2011211344812774 0.25917795779347763
epoch: 7
epoch: 7 loss: 0.1914304091746053
eval loss: 0.19637189120668325 0.2529258329878656
epoch: 8
epoch: 8 loss: 0.18519963572834536
eval loss: 0.19924409863732606 0.25630250123201026
epoch: 9
epoch: 9 loss: 0.1812381279106964
eval loss: 0.19406353672905918 0.25027835370694246
epoch: 10
epoch: 10 loss: 0.17768514178151032
eval loss: 0.1890

epoch: 0
epoch: 0 loss: 0.310226260508905
eval loss: 0.2514518866670896 0.32961218724860325
epoch: 1
epoch: 1 loss: 0.24414612110396985
eval loss: 0.23389236728011936 0.3080934865322685
epoch: 2
epoch: 2 loss: 0.22939282824168108
eval loss: 0.22427321548539156 0.29412031959266566
epoch: 3
epoch: 3 loss: 0.2191414780945405
eval loss: 0.20972219089408478 0.274651320662667
epoch: 4
epoch: 4 loss: 0.21216965815467848
eval loss: 0.2069329761537738 0.27204770324395733
epoch: 5
epoch: 5 loss: 0.2035216560373233
eval loss: 0.19517689344768058 0.2567087318395632
epoch: 6
epoch: 6 loss: 0.1964496449216966
eval loss: 0.19332218477581387 0.25352947742567256
epoch: 7
epoch: 7 loss: 0.19172860494226143
eval loss: 0.19181114299454613 0.2517190581205987
epoch: 8
epoch: 8 loss: 0.1869249697989473
eval loss: 0.18714358641484047 0.2462720449987942
epoch: 9
epoch: 9 loss: 0.18122389138094608
eval loss: 0.1839798669761869 0.24169713007804255
epoch: 10
epoch: 10 loss: 0.17889261685816057
eval loss: 0.180437

epoch: 0
epoch: 0 loss: 0.30546837478859035
eval loss: 0.2479065190363147 0.3298589328834968
epoch: 1
epoch: 1 loss: 0.2410906383735662
eval loss: 0.22903548688422765 0.3045369147102053
epoch: 2
epoch: 2 loss: 0.22466985172421164
eval loss: 0.21530890356138196 0.28920143039824725
epoch: 3
epoch: 3 loss: 0.21436164597609644
eval loss: 0.20767015834358377 0.2775026967587008
epoch: 4
epoch: 4 loss: 0.2065777166638862
eval loss: 0.20051194484358575 0.2685480522210761
epoch: 5
epoch: 5 loss: 0.1979761714371368
eval loss: 0.19105867257636086 0.2560721147027294
epoch: 6
epoch: 6 loss: 0.19233154726017054
eval loss: 0.18787110912509353 0.24959494065707574
epoch: 7
epoch: 7 loss: 0.18796205915679792
eval loss: 0.18534256264468868 0.24659539435977362
epoch: 8
epoch: 8 loss: 0.18181370770071562
eval loss: 0.17711766996975203 0.23587189483315002
epoch: 9
epoch: 9 loss: 0.17889293721404237
eval loss: 0.1741175696599614 0.23199822103913498
epoch: 10
epoch: 10 loss: 0.1762902345908101
eval loss: 0.17

epoch: 0
epoch: 0 loss: 0.31101039585022194
eval loss: 0.262480388404271 0.3442196969965011
epoch: 1
epoch: 1 loss: 0.2453481405001155
eval loss: 0.2327927170854536 0.30681058979780446
epoch: 2
epoch: 2 loss: 0.22851483285042118
eval loss: 0.225529119248756 0.2962588315774305
epoch: 3
epoch: 3 loss: 0.21867257976043344
eval loss: 0.20950096062580717 0.27568725456186677
epoch: 4
epoch: 4 loss: 0.21133759461655136
eval loss: 0.203402523210617 0.2689739051362342
epoch: 5
epoch: 5 loss: 0.2021030159730887
eval loss: 0.1978799859294897 0.2615655632641966
epoch: 6
epoch: 6 loss: 0.19655254141729891
eval loss: 0.18942013824976536 0.2499562263040228
epoch: 7
epoch: 7 loss: 0.19201438484456745
eval loss: 0.19212440016020615 0.25383997625155375
epoch: 8
epoch: 8 loss: 0.18903788716910055
eval loss: 0.1927055394159749 0.2554651273155739
epoch: 9
epoch: 9 loss: 0.18222206633229227
eval loss: 0.1789254869035535 0.23637435392224332
epoch: 10
epoch: 10 loss: 0.17852934427803596
eval loss: 0.182142403

In [11]:
def predict_batch(model, data, device):
    # batch x seq_len x target_size
    with torch.no_grad():
        pred = model(data["sequence"].to(device), data["bpp"].to(device))
        pred = pred.detach().cpu().numpy()
    return_values = []
    ids = data["ids"]
    for idx, p in enumerate(pred):
        id_ = ids[idx]
        assert p.shape == (model.pred_len, len(target_cols))
        for seqpos, val in enumerate(p):
            assert len(val) == len(target_cols)
            dic = {key: val for key, val in zip(target_cols, val)}
            dic["id_seqpos"] = f"{id_}_{seqpos}"
            return_values.append(dic)
    return return_values


def predict_data(model, loader, device, batch_size):
    data_list = []
    for i, data in enumerate(progress_bar(loader)):
        data_list += predict_batch(model, data, device)
    expected_length = model.pred_len * len(loader) * batch_size
    assert len(data_list) == expected_length, f"len = {len(data_list)} expected = {expected_length}"
    return data_list


### Prediction

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()
pub_loader = create_loader(public_df, BATCH_SIZE, is_test=True)
pri_loader = create_loader(private_df, BATCH_SIZE, is_test=True)
pred_df_list = []
c = 0
for fold in range(5):
    model_load_path = f"./model_prediction/model-{fold}.pt"
    ae_model0 = AEModel()
    ae_model1 = AEModel()
    model_pub = FromAeModel(pred_len=107, seq=ae_model0.seq)
    model_pub = model_pub.to(device)
    model_pri = FromAeModel(pred_len=130, seq=ae_model1.seq)
    model_pri = model_pri.to(device)
    state_dict = torch.load(model_load_path, map_location=device)
    model_pub.load_state_dict(state_dict)
    model_pri.load_state_dict(state_dict)
    del state_dict

    data_list = []
    data_list += predict_data(model_pub, pub_loader, device, BATCH_SIZE)
    data_list += predict_data(model_pri, pri_loader, device, BATCH_SIZE)
    pred_df = pd.DataFrame(data_list, columns=["id_seqpos"] + target_cols)
    print(pred_df.head())
    print(pred_df.tail())
    pred_df_list.append(pred_df)
    c += 1
data_dic = dict(id_seqpos=pred_df_list[0]["id_seqpos"])
for col in target_cols:
    vals = np.zeros(pred_df_list[0][col].shape[0])
    for df in pred_df_list:
        vals += df[col].values
    data_dic[col] = vals / float(c)
pred_df_avg = pd.DataFrame(data_dic, columns=["id_seqpos"] + target_cols)
print(pred_df_avg.head())
pred_df_avg.to_csv("./submission.csv", index=False)


public_df: (629, 7)
private_df: (3005, 7)
(1, 107, 107, 3)
(1, 130, 130, 3)


        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.763893     0.556688  1.923611    0.522475  0.866247
1  id_00073f8be_1    2.356476     3.424779  4.201050    3.439347  2.940750
2  id_00073f8be_2    1.681369     0.631982  0.685405    0.781921  0.845763
3  id_00073f8be_3    1.322270     1.198778  1.057178    1.698231  1.852643
4  id_00073f8be_4    0.832696     0.493484  0.475626    0.820543  0.984738
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.029238     0.415154  0.430491    0.562653   
457949  id_ffda94f24_126    0.201090     0.220439  0.521216    0.416186   
457950  id_ffda94f24_127    0.316851     0.057637  0.185643    0.109452   
457951  id_ffda94f24_128   -0.022567     0.230828  0.119131    0.341247   
457952  id_ffda94f24_129   -0.001645     0.314138  0.354411    0.429312   

         deg_50C  
457948  0.289951  
457949  0.444609  
457950  0.477755  
457951  0.245910  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.682296     0.562401  1.711305    0.495677  0.751767
1  id_00073f8be_1    2.401884     3.102218  3.687432    3.032487  2.802202
2  id_00073f8be_2    1.790095     0.918801  0.896496    0.939247  0.958421
3  id_00073f8be_3    1.331566     1.080962  1.020827    1.659496  1.933061
4  id_00073f8be_4    0.918200     0.683522  0.553350    0.949269  0.885069
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.309713     0.401397  0.530908    0.572447   
457949  id_ffda94f24_126    0.533930     0.880228  1.199248    0.963125   
457950  id_ffda94f24_127    0.736162     0.377038  0.606222    0.492867   
457951  id_ffda94f24_128    0.142392     0.220305  0.445076    0.397288   
457952  id_ffda94f24_129    0.135349     0.151642  0.281852    0.250062   

         deg_50C  
457948  0.412012  
457949  0.766519  
457950  0.783108  
457951  0.272773  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.873882     0.629328  1.913900    0.442157  0.626088
1  id_00073f8be_1    2.439712     3.125454  4.556205    3.061587  2.632362
2  id_00073f8be_2    1.735773     0.655750  0.838065    0.841587  0.821964
3  id_00073f8be_3    1.490122     1.112941  1.149845    1.632966  1.822515
4  id_00073f8be_4    0.929349     0.567644  0.458717    0.840934  0.919959
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.102219     0.198818  0.561504    0.383394   
457949  id_ffda94f24_126    0.426389     0.645472  1.597124    0.732918   
457950  id_ffda94f24_127    0.541759    -0.014161  0.267029    0.263714   
457951  id_ffda94f24_128    0.183789    -0.018782  0.114433    0.154522   
457952  id_ffda94f24_129    0.215149     0.443195  0.530809    0.532037   

         deg_50C  
457948  0.266798  
457949  0.695366  
457950  0.414305  
457951  0.048912  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.859874     0.589933  1.708957    0.472692  0.725891
1  id_00073f8be_1    2.305261     3.321405  4.283469    3.250910  2.801708
2  id_00073f8be_2    1.471391     0.751171  0.784935    0.756371  0.584594
3  id_00073f8be_3    1.311311     1.226161  1.338253    1.803424  1.994962
4  id_00073f8be_4    0.942700     0.590930  0.489854    0.934749  0.935799
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.211244     0.278005  0.372369    0.505442   
457949  id_ffda94f24_126    0.279159     0.400880  0.880988    0.755246   
457950  id_ffda94f24_127    0.471423     0.105366  0.312424    0.396284   
457951  id_ffda94f24_128    0.138079     0.198613  0.168048    0.307083   
457952  id_ffda94f24_129    0.157937     0.357426  0.173449    0.259850   

         deg_50C  
457948  0.246365  
457949  0.630982  
457950  0.404413  
457951  0.168317  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.805613     0.682861  2.018642    0.486166  0.867612
1  id_00073f8be_1    2.200093     2.965174  4.797538    2.757437  3.220956
2  id_00073f8be_2    1.661813     0.621131  0.692363    0.703550  0.781266
3  id_00073f8be_3    1.186409     0.850135  1.141097    1.425516  1.710888
4  id_00073f8be_4    0.713717     0.495167  0.458090    0.723052  0.838194
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125   -0.000727     0.251386  0.250382    0.514130   
457949  id_ffda94f24_126    0.210741     0.394590  0.418679    0.632697   
457950  id_ffda94f24_127    0.589247     0.119749  0.137364    0.344920   
457951  id_ffda94f24_128    0.046762     0.236416  0.047719    0.313606   
457952  id_ffda94f24_129    0.058435     0.524227  0.201514    0.576868   

         deg_50C  
457948  0.228026  
457949  0.664528  
457950  0.487069  
457951  0.062943  
4579